# A Python Implementation of GoofyCoin - a simple and useless cryptocurrency

<b>Author:</b> Thiago Cardoso <br>
<b>Created on:</b> 26/03/2021 

## 3 - Implementation Code

### 3.1 - Library Import

In [263]:
import random
import string
import hashlib

from ecdsa import SigningKey

### 3.2 - Our random string generator

In [264]:
# Defining a function to generate a random string with length n.
#
#    Note: There are probably more randomized algo's for generating a random n length str. 
#    I'm employing here a very simple one

def get_random_string(length):
    hexa = string.hexdigits
    result_str = ''.join(random.choice(hexa) for i in range(length))
    return result_str 

### 3.3 - Creating Goofy Keys

In [265]:
# We are going to create the real Goofy Signature and Public Keys outside our GoofyCoin class. 
# I'm using here the ECDSA library (and algo) to generate and further verify Goofy private signature
# For more information about ECDSA algo:
# https://www.cs.miami.edu/home/burt/learning/Csc609.142/ecdsa-cert.pdf

realGoofyKey = SigningKey.generate()     
goofyVerifyingKey = realGoofyKey.verifying_key

### 3.4 - The GoofyCoin Class

In [266]:
class GoofyCoin(object):
    
    def __init__(self):
        self.chain = []
        
    def CreateUser(self):
        userSk = SigningKey.generate()     
        userPk = userSk.verifying_key
        return userSk, userPk   
            
    def CreateCoin(self, goofyKey):
        if goofyKey == realGoofyKey:
            # 1st we gen a random string with 256 chars
            temp_uniqueId = get_random_string(256)
            # 2nd we create a message concatenating Create Coin and the random unique Id. 
            # Ecdsa require using bytes instead of str
            message = bytes("Create Coin " + str(temp_uniqueId), 'utf-8')
            # 3rd we compute the digital signature of this string with Goofy Signature
            signature = goofyKey.sign_deterministic(message)
            # 4th we create a new Goofy Coin object. This object indicates:
            # 'Id': it's unique and random 256 char Id
            # 'Signature': Goofy signature for this specific coin
            # 'Owner': The current owner of this coin. Since we are creating a brand new coin, Goofy owe it
            newGoofyCoin = {
                'Signature': signature,
                'Data': message,
                'Destiny': goofyVerifyingKey,
                'HashPointer': "genesis"
            }
            # 5th we add the new coin to our chain
            self.chain.append(newGoofyCoin)

        else:
            raise Exception("Do not try to fool me")

        
    def TransferCoin(self, destinyPk, userSk, coinSignature):
        
        m = hashlib.sha256()
        m.update(coinSignature)

        # 1st lets find the coin (represented by its signature) in the chain
        for i in self.chain:

            if coinSignature == i.get('Signature'):
                # 2nd lets insert a new message in the chain signaling the coin transfer. 
                # We wont verify if the coin is valid now
                message =  bytes('Pay' + str(destinyPk) + str(m.digest()), 'utf-8')
                signature = userSk.sign_deterministic(message)
                self.chain.append({
                    'Signature': signature,
                    'Data': message,
                    'Destiny': destinyPk,
                    'HashPointer': m.digest()
                })
                return self.chain
        
        raise Exception("This coin do not exist")
        
    def VerifyCoin(self, coin):

        coinSignature = coin.get('Signature')
        coinMessage = coin.get('Data')
        coinHashPointer = coin.get('HashPointer')
        
        # 1st we are going to loop backward through our chain and look for a coin 
        # which hash(signature) is equal to the HashPointer of this 'coin'.
        currentPos = self.chain.index(coin)-1

        while currentPos >= 0:
            
            m = hashlib.sha256()
            m.update(self.chain[currentPos].get('Signature'))
           
            if (coinHashPointer == m.digest()):
                
                # 2sd now that we found the previous coin, we are going to check 
                # if 'coin' was really signed by the destiny of its previous owner.
                self.chain[currentPos].get('Destiny').verify(coinSignature, coinMessage)
                coinSignature = self.chain[currentPos].get('Signature')
                coinMessage = self.chain[currentPos].get('Data')
                coinHashPointer = self.chain[currentPos].get('HashPointer')
                
                # 3rd if 'coin' is ok we are moving to check its previous until we reach 
                # the first in its chain route.
                currentPos = currentPos - 1
                    
            else:
                currentPos = currentPos - 1

        return print('Good Coin')


## 4 - Testing our Coin

In [268]:
Goofy = GoofyCoin()

# Creating a GoofyCoin
Goofy.CreateCoin(realGoofyKey)

# Creating Max, our new GoolfyCoin user
MaxSk, MaxPk = Goofy.CreateUser()

# Transfering a brand new coin to Max
Goofy.TransferCoin(MaxPk, realGoofyKey, Goofy.chain[0].get('Signature'))

# Transfering that coin back to Goofy from Max
MaxKey = SigningKey.generate()
Goofy.TransferCoin(goofyVerifyingKey, MaxSk, Goofy.chain[1].get('Signature'))

# Finally lets make Max transfer to himself a coin he do not owl 
Goofy.TransferCoin(MaxPk, MaxSk, Goofy.chain[2].get('Signature'))

# Test 1: Verifying a brand new coin
Goofy.VerifyCoin(Goofy.chain[0])

# Test 2: Trying to transfer an inexistent coin
Goofy.TransferCoin(MaxPk, MaxSk, b"i dont't exist")

# Test 3: Trying to create a new coin without Goofy Secret Key
Goofy.CreateCoin(MaxSk)

# Test 4: Verifying a coin that was already transfered once
Goofy.VerifyCoin(Goofy.chain[1])

# Test 5: Verifying a coin that was already transfered twice
Goofy.VerifyCoin(Goofy.chain[2])

# Test 6: Verifying a fake coin transfered by Max
Goofy.VerifyCoin(Goofy.chain[3])

[{'Signature': b"\xbc\xa7\x8a\xf4\x80\x0e:\xb6\xd5\x8eg\x8d\xee\x15+\x14\xd2\xde'\xe4 B\xe9\x96\xa1k\xd9\xe6\x97:L\xec\x822s\x103\x8dbr>\xa3\xc3\xcf\xee\x7f\xe4f",
  'Data': b'Create Coin 80dcfF81A5DE000dCe1accfB5b7cF95F52D84eC801509bb49BCbdB3Fbe1fFAdf3013653880D614768F31C49C523D035CaAF14CFC0196FEFeDE7A1EdCA002d875A9B1086f4FE976dF202228Aa15Eb3550eED1559eE73EB6EB5e9eF6e9576963Da1BCFeF6cBe26d2fC4B4E21Fc7B47AEFFdE950fE9b4Fe61Df9bE1cd25a92a9e6F',
  'Destiny': VerifyingKey.from_string(b'\x03o\x7f\xc8\xc0:\x98\x0f\x87Fy\x01\xf8f\\B|:Y\xc3k\x11\xc2\xcb\xdb', NIST192p, sha1),
  'HashPointer': 'genesis'},
 {'Signature': b'\xea\xbej\xab\xf7\xaet(7\xe8\xa9`\xde\xe4\xf7c] \xc8\xd4.&\xb7\x90\xc4\xc9\xc2\xed\xfe\x8b\xc5\x9c\x9a\xbe\xcc\xd1\xd5w\xf8\x8f\xa1\xc8\xb9\xd2\xab4\xc0\xb3',
  'Data': b'PayVerifyingKey.from_string(b\'\\x03\\x90\\xfb\\xa3\\xce\\x98\\xe9>/"\\x9b\\xa4\\x1b\\x891\\xf0Y\\x10;GS\\x93\\xb1[n\', NIST192p, sha1)b\'\\x95}@c\\x0b\\x88b\\\\2\\xc3\\xad\\x99\\xe9M\\x1a\\x80YBw\\xbf\\xa1\\